Kaitlin Helfter, CPSC 4770 Distributed and Cluster Computing, Assignment 7. Due November 27th.

I worked alone on this project, only using class resources. This code uses Apache Spark to go through a series of 1000 Google trace files (500 job_event csv files and 500 task_usage csv files) and pull the necessary information from each directory to fulfill the requirements for the assignment.

The desciptions of each part are below every heading of parts 1-4.

In [ ]:
!module list

This ensures I can run code on Palmetto

In [1]:
!cypress-kinit
!klist

Ticket cache: FILE:/home/khelfte/.krb5cc
Default principal: khelfte@PALMETTO.CLEMSON.EDU

Valid starting       Expires              Service principal
12/02/2017 17:15:18  12/09/2017 17:15:18  krbtgt/PALMETTO.CLEMSON.EDU@PALMETTO.CLEMSON.EDU


**PART 1**
Create the two directories named job_events and task_usage under your home directory on HDFS and then store the decompressed job_events and task_usage files into these respective directories.

In [ ]:
!hdfs dfs -ls job_events/job_events

In [ ]:
!hdfs dfs -ls task_usage/task_usage

In [4]:
!hdfs dfs -cat job_events/job_events/part-00000-of-00500.csv \
    2>/dev/null | head -n 5

0,,3418309,0,70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,3,IHgtoxEBuUTHNbUeVs4hzptMY4n8rZKLbZg+Jh5fNG4=,wAmgn2H74cdoMuSFwJF3NaUEaudVBTZ0/HaNZBwIpEQ=
0,,3418314,0,70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,3,L52XDyhi9x9ChmVBZ1qavOFmnzPeVsvQ2QyGmBZcV4s=,ShNjeaoUeqGV2i9WMKEX9HTeuc9K2Fdfovibt7Mp6qI=
0,,3418319,0,70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,3,vq0IN3BWEbkDjYgYvkrVyH6OWoUoDwFFf3j/syEZzLA=,1A2GM17AzHRcKJcJet/oIF7FOORyFcAOcUSpR9Fqou8=
0,,3418324,0,70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,3,X+Vce15Yu3BCKb7Ttc6hvINAzdfG3NtYEDNNsPdMGKo=,seczVo7MBfi/kH3+eH/40Hxs0l2YKFtXFnamcvMjSQI=
0,,3418329,0,70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,3,EeK3DUWYi1P0vgBTp7wZdUos8UKj/+/FqudTLohMQ9M=,OEeQsaVr4kdGHFwQ2liQlDZ18529HEMmu6B6/3K1cBA=


In [5]:
!hdfs dfs -cat  task_usage/task_usage/part-00000-of-00500.csv \
    2>/dev/null | head -n 5

600000000,900000000,3418309,0,4155527081,0.001562,0.06787,0.07568,0.001156,0.001503,0.06787,2.861e-06,0.0001869,0.03967,0.0003567,2.445,0.007243,0,1,0
600000000,900000000,3418309,1,329150663,0.001568,0.06787,0.07556,0.0003195,0.0007,0.06787,5.722e-06,0.0001879,0.03302,0.0009289,2.1,0.005791,0,1,0
600000000,900000000,3418314,0,3938719206,0.0003071,0.08044,0.09521,0.0002823,0.0006704,0.08044,4.768e-06,0.0001841,0.02377,0.0007858,5.588,0.0208,0,1,0
600000000,900000000,3418314,1,351618647,0.0003004,0.08044,0.09521,0.0005369,0.0008698,0.08044,9.537e-06,0.0001831,0.007919,0.002285,5.198,0.02038,0,1,0
600000000,900000000,3418319,0,431052910,0.0004606,0.07715,0.0874,0.0006247,0.0008516,0.07715,1.907e-06,0.0002041,0.005112,0.0002146,2.937,0.009449,0,1,0


**PART 2**
Calculate the total run time from SUBMIT to either EVICT, FAIL, FINISH, KILL, or LOST for each job (job_events) and return only the top 20 longest run time jobs. The printout should use a tab-separated format that includes jobID, the total run time, and the job's final status (EVICT, FAIL, FINISH, KILL, or LOST). Run this job inside your notebook and print out the result of this job .

In [27]:
import sys
import os

sys.path.insert(0, '/usr/hdp/current/spark2-client/python')
sys.path.insert(0, '/usr/hdp/current/spark2-client/python/lib/py4j-0.10.4-src.zip')

os.environ['SPARK_HOME'] = '/usr/hdp/current/spark2-client/'
os.environ['SPARK_CONF_DIR'] = '/etc/hadoop/synced_conf/spark2/'
os.environ['PYSPARK_PYTHON'] = '/software/anaconda3/4.2.0/bin/python'

import pyspark
conf = pyspark.SparkConf()
conf.setMaster("yarn")
conf.set("spark.driver.memory","4g")
conf.set("spark.executor.memory","60g")
conf.set("spark.num.executors","3")
conf.set("spark.executor.cores","12")

sc = pyspark.SparkContext(conf=conf)

In [8]:
part2 = sc.textFile("job_events/job_events/")

In [9]:
part2.take(5)

['0,,3418309,0,70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,3,IHgtoxEBuUTHNbUeVs4hzptMY4n8rZKLbZg+Jh5fNG4=,wAmgn2H74cdoMuSFwJF3NaUEaudVBTZ0/HaNZBwIpEQ=',
 '0,,3418314,0,70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,3,L52XDyhi9x9ChmVBZ1qavOFmnzPeVsvQ2QyGmBZcV4s=,ShNjeaoUeqGV2i9WMKEX9HTeuc9K2Fdfovibt7Mp6qI=',
 '0,,3418319,0,70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,3,vq0IN3BWEbkDjYgYvkrVyH6OWoUoDwFFf3j/syEZzLA=,1A2GM17AzHRcKJcJet/oIF7FOORyFcAOcUSpR9Fqou8=',
 '0,,3418324,0,70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,3,X+Vce15Yu3BCKb7Ttc6hvINAzdfG3NtYEDNNsPdMGKo=,seczVo7MBfi/kH3+eH/40Hxs0l2YKFtXFnamcvMjSQI=',
 '0,,3418329,0,70s3v5qRyCO/1PCdI6fVXnrW8FU/w+5CKRSa72xgcIo=,3,EeK3DUWYi1P0vgBTp7wZdUos8UKj/+/FqudTLohMQ9M=,OEeQsaVr4kdGHFwQ2liQlDZ18529HEMmu6B6/3K1cBA=']

In [10]:
mappedKeys = part2.map(lambda line: (line.split(",")[2], (line.split(",")[0], line.split(",")[3])))
array = {'1','7','8'}
mappedKeys = mappedKeys.filter(lambda x: x[1][1] not in array)
mappedKeys.count()

1340161

In [11]:
groupKeyIDs = mappedKeys.groupByKey()
groupKeyIDs.take(10)

[('6411883093', <pyspark.resultiterable.ResultIterable at 0x2b05737fb5f8>),
 ('6366070622', <pyspark.resultiterable.ResultIterable at 0x2b05737fb048>),
 ('6453791633', <pyspark.resultiterable.ResultIterable at 0x2b05737fb438>),
 ('6337828535', <pyspark.resultiterable.ResultIterable at 0x2b05737fb400>),
 ('6294094629', <pyspark.resultiterable.ResultIterable at 0x2b05737fb9e8>),
 ('6467417008', <pyspark.resultiterable.ResultIterable at 0x2b05737fb390>),
 ('6413900542', <pyspark.resultiterable.ResultIterable at 0x2b05737fb080>),
 ('6292928584', <pyspark.resultiterable.ResultIterable at 0x2b05737fb3c8>),
 ('6472512110', <pyspark.resultiterable.ResultIterable at 0x2b05737fb4a8>),
 ('6288828883', <pyspark.resultiterable.ResultIterable at 0x2b05737fb198>)]

In [12]:
groupKeyIDs.count()

672074

In [13]:
mapValuesToListIDs = groupKeyIDs.mapValues(list)
mapValuesToListIDs.take(5)

[('6482126368', [('2456309685616', '0'), ('2457477140483', '5')]),
 ('6255633967', [('61489815497', '0'), ('61678306276', '4')]),
 ('6400514925', [('1565800212688', '0'), ('1566359972065', '5')]),
 ('6337828535', [('885404987717', '0'), ('888500163050', '5')]),
 ('6294094629', [('412803442322', '0'), ('413353523886', '4')])]

In [14]:
def totTime(V):
    high = max(V)
    low = min(V)
    return (int(high[0])-int(low[0]),high[1])

avgTime01 = mapValuesToListIDs.mapValues(totTime)
avgTime01.take(5)

[('6411883093', (875878961, '5')),
 ('6366070622', (1097966937, '5')),
 ('6453791633', (252126965, '5')),
 ('6337828535', (3095175333, '5')),
 ('6294094629', (550081564, '4'))]

In [15]:
finalTime = avgTime01.collectAsMap()

In [16]:
status = {'2': 'EVICT', '3': 'FAIL', '4':'FINISH', '5': 'KILL', '6': 'LOST' }
val = 0;
for x in sorted(finalTime, key=lambda x: finalTime.get(x)[0], reverse=True):
    print(repr(x), "\t", repr(finalTime.get(x)[0]), "\t", repr(status.get(finalTime.get(x)[1])))
    val +=1
    if(val == 20):
        break

'6217291782' 	 2491521082981 	 'KILL'
'4867841244' 	 2488059970906 	 'KILL'
'5782542009' 	 2463114365081 	 'KILL'
'6035592839' 	 2421933921466 	 'FAIL'
'6182354041' 	 2340306707627 	 'KILL'
'6182354078' 	 2340306707617 	 'KILL'
'6182353820' 	 2340304587870 	 'KILL'
'6182354160' 	 2340302039296 	 'KILL'
'6182354117' 	 2340302039286 	 'KILL'
'6182353894' 	 2340302038692 	 'KILL'
'6182353971' 	 2340300639012 	 'KILL'
'6182354009' 	 2340300639004 	 'KILL'
'6182353936' 	 2340300638994 	 'KILL'
'6182353704' 	 2340299854097 	 'KILL'
'6182353777' 	 2340299181750 	 'KILL'
'6182353639' 	 2340298451442 	 'KILL'
'6182353667' 	 2340298451053 	 'KILL'
'6182353613' 	 2340297782400 	 'KILL'
'6182353860' 	 2340297020638 	 'KILL'
'6182366148' 	 2340297020143 	 'KILL'


**PART 3** 
Count the number of computing tasks for each unique job (task_usage) and sort the results using total number of computing tasks. Run this job inside your notebook and print out the top 20 lines of the result.

In [17]:
part3 = sc.textFile("task_usage/task_usage/")

In [18]:
taskcount = part3.map(lambda line: ((line.split(",")[2]),1)).reduceByKey(lambda a, b: a+b) 
taskcount.take(20)

[('6486065147', 3),
 ('6377521019', 5),
 ('6365006831', 9081),
 ('6445247513', 123),
 ('6411882224', 27),
 ('6358866175', 314),
 ('6321771899', 5),
 ('6340240734', 7),
 ('6290566424', 4),
 ('6342076041', 5),
 ('6364097655', 8),
 ('6339595271', 13),
 ('6346736462', 27900),
 ('6332705324', 3),
 ('6403077275', 5),
 ('6389962836', 1048),
 ('6255203248', 35),
 ('6305901911', 4),
 ('6391894019', 92),
 ('6277749801', 1)]

In [19]:
finalTask = taskcount.collectAsMap()

In [20]:
val = 0;
for x in sorted(finalTask, key=lambda x: finalTask.get(x), reverse=True):
    print(repr(x), "\t", repr(finalTask.get(x)), )
    val +=1
    if(val == 20):
        break

'6336594489' 	 52619489
'2902878580' 	 36494184
'5402488769' 	 33674510
'6176858948' 	 25954362
'6176871439' 	 25674474
'6114773114' 	 17377322
'6221861800' 	 16646074
'4028922835' 	 16630408
'6419322512' 	 14689441
'4665712499' 	 13511361
'4765556460' 	 10853784
'4665891113' 	 10731174
'6324858588' 	 9081671
'6312699084' 	 8923393
'2902878525' 	 8762679
'6388672237' 	 8597389
'4665892165' 	 8402036
'4476805516' 	 8276477
'6261947650' 	 8004506
'4665903885' 	 6886809


**PART 4**
 Calculate total CPU usage time (CPU-core seconds per second) for each job (task_usage) and sort the results using total CPU usage time from highest to lowest. The printout should use a tab-separated format that includes jobID and the total CPU usage time. Run this job inside your notebook and print out the top 20 lines of the result.

In [28]:
part4 = sc.textFile("task_usage/task_usage/")

In [29]:
def tofloat(val):
    return float(val)

taskusage = part4.map(lambda line: (line.split(",")[2],tofloat(line.split(",")[5])))
taskusage.take(10)

[('3418309', 0.001562),
 ('3418309', 0.001568),
 ('3418314', 0.0003071),
 ('3418314', 0.0003004),
 ('3418319', 0.0004606),
 ('3418319', 0.0005188),
 ('3418324', 0.0003805),
 ('3418324', 0.0003271),
 ('3418329', 0.002518),
 ('3418329', 0.002705)]

In [30]:
groupedUsage = taskusage.groupByKey()

groupedUsage.take(10)


[('6486065147', <pyspark.resultiterable.ResultIterable at 0x2b058eac1550>),
 ('6411882224', <pyspark.resultiterable.ResultIterable at 0x2b058eac1a58>),
 ('6365006831', <pyspark.resultiterable.ResultIterable at 0x2b058eac1b00>),
 ('6419891734', <pyspark.resultiterable.ResultIterable at 0x2b058eac1a90>),
 ('6377521019', <pyspark.resultiterable.ResultIterable at 0x2b058eac1b70>),
 ('6358866175', <pyspark.resultiterable.ResultIterable at 0x2b058eac1ac8>),
 ('6322941134', <pyspark.resultiterable.ResultIterable at 0x2b058eac1be0>),
 ('6340240734', <pyspark.resultiterable.ResultIterable at 0x2b058eac1c18>),
 ('6290566424', <pyspark.resultiterable.ResultIterable at 0x2b058eac1c50>),
 ('6342076041', <pyspark.resultiterable.ResultIterable at 0x2b058eac1b38>)]

In [31]:
listUsage = groupedUsage.mapValues(list)
listUsage.take(20)

[('6486065147', [0.000145, 0.0003633, 0.0]),
 ('6411882224',
  [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0192,
   0.007774,
   0.0123,
   0.01385,
   0.009933,
   0.0002565,
   0.001625,
   0.0,
   0.005875,
   0.0001307,
   3.719e-05,
   0.0,
   0.004562,
   0.001268,
   0.001242,
   4.196e-05,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0]),
 ('6365006831',
  [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0

In [32]:
summedUsage = listUsage.mapValues(lambda V: sum(V))
summedUsage.take(20)

[('6486065147', 0.0005082999999999999),
 ('6411882224', 0.07809535000000002),
 ('6365006831', 177.13980506699974),
 ('6419891734', 0.0213076),
 ('6377521019', 0.033139800000000004),
 ('6358866175', 1.0323301000000003),
 ('6322941134', 0.0200562),
 ('6340240734', 0.00132846),
 ('6290566424', 0.0050463),
 ('6342076041', 0.0062598),
 ('6364097655', 0.0390915),
 ('6339595271', 0.043679399999999986),
 ('6346736462', 17.35479736999967),
 ('6332705324', 6.676e-05),
 ('6403077275', 0.0005360499999999999),
 ('6389962836', 17.802822130000006),
 ('6255203248', 0.13005790000000003),
 ('6305901911', 0.0038741),
 ('6391894019', 0.21089853999999997),
 ('6277749801', 0.0002241)]

In [33]:
finalUsage = summedUsage.collectAsMap()
val = 0;
for x in sorted(finalUsage, key=lambda x: finalUsage.get(x), reverse=True):
    print(repr(x), "\t", repr(finalUsage.get(x)), )
    val +=1
    if(val == 20):
        break

'6419322512' 	 1687821.4698673268
'4028922835' 	 932825.8257848633
'4665712499' 	 677763.0328915882
'6282133822' 	 643076.3033474556
'4665897969' 	 615944.6306261823
'6399142719' 	 570758.1227292098
'6119300167' 	 530648.8996312444
'4765556460' 	 476376.6776045609
'4665617026' 	 471178.8223110897
'2902878580' 	 354327.0122481051
'5022579678' 	 339772.92600751424
'3998352223' 	 293703.467764781
'6184860354' 	 268556.4971444907
'3244832720' 	 241762.30491264982
'6295210420' 	 230658.9302296796
'5544435560' 	 227942.35933412035
'6336594489' 	 176164.84001242716
'5456984290' 	 174002.67624088554
'5544436431' 	 173067.35213118012
'3695027940' 	 172468.02094969273


In [26]:
sc.stop()